# Python HANA ML APL

<div class="alert alert-block alert-info">
<b>Building a Predictive Model for Insurance Fraud Detection.</b> <br>
</div>

## Learn from historical Insurance Claims

### Create an HANA Dataframe for the training data

In [ ]:
from hana_ml import dataframe as hd
conn = hd.ConnectionContext(userkey='MLMDA_KEY')
sql_cmd = 'SELECT * FROM "APL_SAMPLES"."AUTO_CLAIMS_FRAUD" ORDER BY CLAIM_ID'
hdf_train = hd.DataFrame(conn, sql_cmd)

In [ ]:
hdf_train.head(6).collect()

### Fit with APL Gradient Boosting

In [ ]:
from hana_ml.algorithms.apl.gradient_boosting_classification import GradientBoostingBinaryClassifier
apl_model = GradientBoostingBinaryClassifier(variable_auto_selection = True)
apl_model.fit(hdf_train, label='IS_FRAUD', key='CLAIM_ID')

##### Model Reports

In [ ]:
from hana_ml.visualizers.unified_report import UnifiedReport
UnifiedReport(apl_model).build().display()

In [ ]:
df = apl_model.get_debrief_report('ClassificationRegression_VariablesExclusion').collect()
df = df[['Variable', 'Reason For Exclusion']]
df.style.hide(axis='index')

In [ ]:
my_filter = "\"Partition\" = 'Validation' and \"Indicator\" in ('AUC','F1 Score','Cohen''s kappa')"
df = apl_model.get_debrief_report('ClassificationRegression_Performance').filter(my_filter).collect()
df.drop('Oid', axis=1, inplace=True)
format_dict = {'Value':'{:,.3f}'}
df.style.format(format_dict).hide(axis='index')

In [ ]:
my_filter = "\"Partition\"='Validation'"
df = apl_model.get_debrief_report('BinaryTarget_Statistics').filter(my_filter).collect()
df.drop('Oid', axis=1, inplace=True)
format_dict = {'% Positive Weight':'{:,.1f}%', '% Negative Weight':'{:,.1f}%', 'Weight':'{:,.0f}'}
df.style.format(format_dict).hide(axis='index')

### Save the Trained Model

In [ ]:
from hana_ml.model_storage import ModelStorage
model_storage = ModelStorage(connection_context=conn, schema='USER_APL')
apl_model.name = 'My Fraud Model'  
model_storage.save_model(model=apl_model, if_exists='replace')

In [ ]:
model_storage.list_models()